# Proyecto Final: Los Michis

In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics, svm
import matplotlib.pyplot as plt

# Configuración de pandas para mostrar todas las filas
pd.set_option('display.max_rows', None)


### Recolección de Datos.
### Preprocesamiento de Datos.


In [47]:
# Carga de los datasets
df0 = pd.read_csv('/Users/danny/Library/Mobile Documents/com~apple~CloudDocs/Documents/7mo Semestre/Mineria de datos/Ejemplos/ProyectoFinal/Datasets/ensadul2022_entrega_w.csv', delimiter=';')
df1 = pd.read_csv('/Users/danny/Library/Mobile Documents/com~apple~CloudDocs/Documents/7mo Semestre/Mineria de datos/Ejemplos/ProyectoFinal/Datasets/ensaantro2022_entrega_w.csv', delimiter=';')
df2 = pd.read_csv('/Users/danny/Library/Mobile Documents/com~apple~CloudDocs/Documents/7mo Semestre/Mineria de datos/Ejemplos/ProyectoFinal/Datasets/Determinaciones_bioquímicas_cronicas_deficiencias_9feb23.csv', delimiter=';')
df3 = pd.read_csv('/Users/danny/Library/Mobile Documents/com~apple~CloudDocs/Documents/7mo Semestre/Mineria de datos/Ejemplos/ProyectoFinal/Datasets/ensafisica2022_adultos_entrega_w.csv', delimiter=';')
# Selección de columnas de interés
df0_selected_columns = [
    'FOLIO_INT','a0301'
]

df1_selected_columns = ['FOLIO_INT',
    'an01_1','an01_2','an04_1','an04_2','an08_1','an08_2','an10','an12_1','an12_2','an15_1','an15_2','an17_1',
    'an21_1','an21_2','an23','an25','an27_01s','an27_01d','an27_02s','an27_02d','an27_03s','an27_03d','an29',
]

df2_selected_columns = [    
    'FOLIO_I', 'FOLIO_INT', 'entidad', 'municipio', 'h0302', 'h0303', 'fech_nac',
    'sc', 'sh', 'sm', 'int_hemoglobina', 'nota11', 'nota02', 'hb02', 'hb07a',
    'completa', 'MATRIZ_SUERO', 'valor_AC_URICO', 'valor_ALBU','valor_COL_HDL',
    'valor_COL_LDL','valor_COLEST','valor_CREAT','valor_GLU_SUERO','valor_INSULINA',
    'valor_PCR','valor_TRIG','MATRIZ_SANGRE', 'valor_EAG','valor_HB1AC','MATRIZ_SUEROMICRO',
    'valor_FERRITINA','valor_FOL','valor_HCST','valor_PROTCREAC','valor_STFR_FEB23',
    'valor_VIT_B12','valor_VIT_D','ponde_v','cuenta_mic', 'ponde_m12a49', 'ponde_m0a11', 'estrato'
]

df3_selected_columns = ['FOLIO_INT',
    'fa0400','fa0401','fa0403','fa0405','fa0406bh','fa0406bm','fa0407h',
    'fa0408','fa0409h','fa0409m','fa0410','fa0411','fa0412','fa0413',
    'fa0414','fa0415',
]

# Union de los datasets por el campo FOLIO_INT 
merged_df = pd.merge(df0[df0_selected_columns], df3[df3_selected_columns], on='FOLIO_INT', how='inner')
merged_df = pd.merge(merged_df, df2[df2_selected_columns], on='FOLIO_INT', how='inner')
merged_df = pd.merge(merged_df, df1[df1_selected_columns], on='FOLIO_INT', how='inner')

merged_df.drop(merged_df[merged_df['a0301'] == 2].index, inplace=True)

#limpieza de datos
merged_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
null_count = merged_df.isnull().sum()
print(null_count)

# Nuevo dataset
merged_df.to_csv('/Users/danny/Library/Mobile Documents/com~apple~CloudDocs/Documents/7mo Semestre/Mineria de datos/Ejemplos/ProyectoFinal/Datasets/Pd.csv', index=False)



/var/folders/zh/1___606s7jv0cx79lmmsklq80000gn/T/ipykernel_11086/3343144620.py:2: DtypeWarning: Columns (570) have mixed types. Specify dtype option on import or set low_memory=False.
  df0 = pd.read_csv('/Users/danny/Library/Mobile Documents/com~apple~CloudDocs/Documents/7mo Semestre/Mineria de datos/Ejemplos/ProyectoFinal/Datasets/ensadul2022_entrega_w.csv', delimiter=';')
/var/folders/zh/1___606s7jv0cx79lmmsklq80000gn/T/ipykernel_11086/3343144620.py:5: DtypeWarning: Columns (90,91,96,97,102,103,104,107,108,111,112,113,114,115,116,117,118) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('/Users/danny/Library/Mobile Documents/com~apple~CloudDocs/Documents/7mo Semestre/Mineria de datos/Ejemplos/ProyectoFinal/Datasets/ensafisica2022_adultos_entrega_w.csv', delimiter=';')


FOLIO_INT               0
a0301                   0
fa0400                  0
fa0401                  0
fa0403                 14
fa0405                 14
fa0406bh             4926
fa0406bm             4943
fa0407h                14
fa0408                 14
fa0409h              1011
fa0409m              1019
fa0410                 14
fa0411                 14
fa0412                 14
fa0413                 14
fa0414                 14
fa0415                 14
FOLIO_I                 0
entidad                 0
municipio               0
h0302                   0
h0303                   0
fech_nac              256
sc                    256
sh                    256
sm                    256
int_hemoglobina       256
nota11                324
nota02               2711
hb02                 3021
hb07a                3224
completa                0
MATRIZ_SUERO            0
valor_AC_URICO       3289
valor_ALBU           3289
valor_COL_HDL        3289
valor_COL_LDL        3289
valor_COLEST

### Normalización/estandarización de los datos.
